In [340]:
import pandas as pd
from gensim import models, corpora
from gensim.parsing.preprocessing import preprocess_string, strip_tags, strip_punctuation
from gensim.parsing.preprocessing import remove_stopwords, stem_text, strip_non_alphanum, strip_multiple_whitespaces
from gensim.parsing.preprocessing import strip_short, strip_numeric
from gensim.parsing.preprocessing import STOPWORDS
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.utils import class_weight
import multiprocessing as mp
import calendar
import nltk
import pandas as pd
import numpy as np
import tensorflow as tf
import parmap
import ast

In [341]:
topics_index_to_name_map = {
    0: 'Agriculture, animals, food and rural affairs',
    1: 'Asylum, immigration and nationality',
    2: 'Business, industry and consumers',
    3: 'Communities and families',
    4: 'Crime, civil law, justice and rights',
    5: 'Culture, media and sport',
    6: 'Defence',
    7: 'Economy and finance',
    8: 'Education',
    9: 'Employment and training',
    10: 'Energy and environment',
    11: 'European Union',
    12: 'Health services and medicine',
    13: 'Housing and planning',
    14: 'International affairs',
    15: 'Parliament, government and politics',
    16: 'Science and technology',
    17: 'Social security and pensions',
    18: 'Social services',
    19: 'Transport',
    20: 'Others'
}
topics_name_to_index_map = {y:x for x,y in topics_index_to_name_map.items()}

def strip_short2(text):
    return strip_short(text, minsize=4)


def preprocess_text(text):
    FILTERS = [lambda x: x.lower(), strip_multiple_whitespaces, strip_tags, strip_punctuation,
                   strip_non_alphanum, strip_numeric, strip_short2]
    return preprocess_string(text, FILTERS)

def preprocess(topic):
    ret = []
    topic = topic.strip()
    
    if '|' in topic:
        topics = topic.split('|')
        t = topics[0]
        t = t.strip()
        return topics_name_to_index_map[t]
        

In [342]:
def contains_european(x):
    y = x.lower()
    flag = 'e.u' in y or 'eu' in y or 'european' in y or 'european union' in y or 'europeanunion' in y
    return flag

In [376]:
year = 2016
bbc_id = 279
exclude = 'no_Others'

In [377]:
df = pd.read_csv('./data/partition_predictions/topics_pred_on_bert_partitioned_bbc_{}_{}_with_news_classifier_{}.csv'.format(bbc_id, year, exclude))
df = df.drop(['Unnamed: 0'], axis=1)
df['topic'] = df['topic'].apply(lambda x: ast.literal_eval(x))
df['topic'] = df['topic'].apply(lambda x: x[0][0])
df['length'] = df['transcript'].apply(lambda x: len(x.split()))
df['european_union'] = df['transcript'].apply(lambda x: contains_european(x))
df

,partition_id,date,transcript,topic,length,european_union
0,0,1-mar-2016,THUDDINGI DRUMMING There is a rhythm that beat...,"Culture, media and sport",104,False
1,1,1-mar-2016,"This is Sky News at 10pm, live from Virginia, ...","Parliament, government and politics",180,False
2,1,1-mar-2016,"Good evening from Virginia, one of 12 states v...","Parliament, government and politics",816,False
3,1,1-mar-2016,Let me just show you what they have been doing...,"Parliament, government and politics",144,False
4,1,1-mar-2016,And we’re expecting that story to feature in t...,"Parliament, government and politics",354,False
...,...,...,...,...,...,...
20631,791,24-oct-2016,Christian bakers who refused to make a gay—fri...,"Parliament, government and politics",122,False
20632,791,24-oct-2016,The operation to close the ‘Jungle’ migrants c...,"Parliament, government and politics",76,True
20633,791,24-oct-2016,"From Fro m befo re From before dawn, they gath...",European Union,710,True
20634,791,24-oct-2016,"A little later, in other A little later, in ot...","Culture, media and sport",78,True


In [367]:
df['partition_id'].value_counts().mean()

22.84957627118644

In [368]:
n_bbc = df['partition_id'].max()
n_bbc

472

In [332]:
topics = list(topics_name_to_index_map.keys())
header = ['day', 'month']
header += topics
rows = []

In [104]:
for month in range(1, 13):
    for day in range(1, calendar.monthrange(year, month)[1]): # calendar.monthrange(year, month)[1]
        date_string = '{}-{}-{}'.format(day, calendar.month_abbr[month].lower(), year)
        print(date_string)
        df_day = df.loc[df.date == date_string]
#         df_day = df_day.loc[df.partition_id == 1259]
        partition_id = None
        pos = 0
        topic_ranks_for_day = {topic:[] for topic in topics}
        last_topic = None
#         display(df_day)
        for index, row in df_day.iterrows():
            if partition_id is None:
                partition_id = row['partition_id']
                pos = 1
                last_topic = row['topic']
            elif partition_id == row['partition_id']:
                if last_topic == row['topic']:
                    continue
                last_topic = row['topic']
                pos += 1
            else:
                pos = 1
                partition_id = row['partition_id']
                last_topic = row['topic']
            topic_ranks_for_day[row['topic']].append(pos)
#         print(topic_ranks_for_day)
        row = [day, month]
        row += [np.mean(topic_ranks_for_day[topic]) for topic in topics]
        rows.append(row)

1-jan-2015
2-jan-2015
3-jan-2015
4-jan-2015
5-jan-2015
6-jan-2015
7-jan-2015
8-jan-2015
9-jan-2015
10-jan-2015
11-jan-2015
12-jan-2015
13-jan-2015
14-jan-2015
15-jan-2015
16-jan-2015
17-jan-2015
18-jan-2015
19-jan-2015
20-jan-2015
21-jan-2015
22-jan-2015
23-jan-2015
24-jan-2015
25-jan-2015
26-jan-2015
27-jan-2015
28-jan-2015
29-jan-2015
30-jan-2015
1-feb-2015
2-feb-2015
3-feb-2015
4-feb-2015
5-feb-2015
6-feb-2015
7-feb-2015
8-feb-2015
9-feb-2015
10-feb-2015
11-feb-2015
12-feb-2015
13-feb-2015
14-feb-2015
15-feb-2015
16-feb-2015
17-feb-2015
18-feb-2015
19-feb-2015
20-feb-2015
21-feb-2015
22-feb-2015
23-feb-2015
24-feb-2015
25-feb-2015
26-feb-2015
27-feb-2015
1-mar-2015
2-mar-2015
3-mar-2015
4-mar-2015
5-mar-2015
6-mar-2015
7-mar-2015
8-mar-2015
9-mar-2015
10-mar-2015
11-mar-2015
12-mar-2015
13-mar-2015
14-mar-2015
15-mar-2015
16-mar-2015
17-mar-2015
18-mar-2015
19-mar-2015
20-mar-2015
21-mar-2015
22-mar-2015
23-mar-2015
24-mar-2015
25-mar-2015
26-mar-2015
27-mar-2015
28-mar-2015
29-mar-

/home/ubuntu/anaconda3/envs/econ/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/ubuntu/anaconda3/envs/econ/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


30-mar-2015
1-apr-2015
2-apr-2015
3-apr-2015
4-apr-2015
5-apr-2015
6-apr-2015
7-apr-2015
8-apr-2015
9-apr-2015
10-apr-2015
11-apr-2015
12-apr-2015
13-apr-2015
14-apr-2015
15-apr-2015
16-apr-2015
17-apr-2015
18-apr-2015
19-apr-2015
20-apr-2015
21-apr-2015
22-apr-2015
23-apr-2015
24-apr-2015
25-apr-2015
26-apr-2015
27-apr-2015
28-apr-2015
29-apr-2015
1-may-2015
2-may-2015
3-may-2015
4-may-2015
5-may-2015
6-may-2015
7-may-2015
8-may-2015
9-may-2015
10-may-2015
11-may-2015
12-may-2015
13-may-2015
14-may-2015
15-may-2015
16-may-2015
17-may-2015
18-may-2015
19-may-2015
20-may-2015
21-may-2015
22-may-2015
23-may-2015
24-may-2015
25-may-2015
26-may-2015
27-may-2015
28-may-2015
29-may-2015
30-may-2015
1-jun-2015
2-jun-2015
3-jun-2015
4-jun-2015
5-jun-2015
6-jun-2015
7-jun-2015
8-jun-2015
9-jun-2015
10-jun-2015
11-jun-2015
12-jun-2015
13-jun-2015
14-jun-2015
15-jun-2015
16-jun-2015
17-jun-2015
18-jun-2015
19-jun-2015
20-jun-2015
21-jun-2015
22-jun-2015
23-jun-2015
24-jun-2015
25-jun-2015
26-jun-

In [37]:
res = pd.DataFrame(rows, columns=header)

In [38]:
res

,day,month,"Agriculture, animals, food and rural affairs","Asylum, immigration and nationality","Business, industry and consumers",Communities and families,"Crime, civil law, justice and rights","Culture, media and sport",Defence,Economy and finance,Education,Employment and training,Energy and environment,European Union,Health services and medicine,Housing and planning,International affairs,"Parliament, government and politics",Science and technology,Social security and pensions,Social services,Transport,Others
0,1,1,39.333333,NaN,128.000000,NaN,47.615385,47.983051,NaN,128.0,NaN,NaN,83.500000,NaN,40.000000,NaN,59.966667,52.296296,NaN,NaN,NaN,42.533333,NaN
1,2,1,118.500000,NaN,46.000000,NaN,41.166667,80.026316,90.375000,NaN,NaN,NaN,95.500000,NaN,91.142857,NaN,76.566667,80.562500,NaN,NaN,NaN,35.666667,NaN
2,3,1,10.333333,NaN,NaN,NaN,11.500000,14.884615,NaN,NaN,NaN,NaN,5.000000,NaN,NaN,NaN,13.571429,14.120000,NaN,NaN,NaN,8.666667,NaN
3,4,1,8.000000,NaN,25.000000,8.5,15.000000,15.115385,NaN,17.0,NaN,NaN,18.000000,44.0,8.800000,NaN,13.916667,14.076923,NaN,NaN,NaN,13.750000,NaN
4,5,1,15.500000,NaN,45.375000,NaN,15.600000,23.893617,3.000000,6.0,NaN,NaN,19.000000,2.0,40.833333,NaN,23.142857,19.078431,NaN,NaN,NaN,26.750000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349,26,12,96.000000,NaN,34.625000,NaN,105.000000,37.133333,NaN,55.0,NaN,NaN,NaN,NaN,17.000000,NaN,47.896552,33.140000,NaN,NaN,NaN,33.875000,NaN
350,27,12,91.666667,NaN,88.000000,NaN,42.250000,63.389831,51.333333,108.0,NaN,NaN,7.000000,NaN,82.000000,NaN,51.902439,62.065217,NaN,NaN,NaN,74.857143,NaN
351,28,12,68.333333,NaN,87.833333,NaN,13.000000,57.280702,NaN,NaN,NaN,NaN,58.000000,NaN,64.307692,NaN,56.733333,58.142857,NaN,NaN,NaN,48.533333,NaN
352,29,12,NaN,NaN,118.000000,NaN,35.375000,58.340426,NaN,NaN,NaN,NaN,41.333333,9.0,61.500000,NaN,54.241379,56.533333,NaN,NaN,NaN,57.545455,NaN


In [39]:
res.to_csv('partition_topic_ranks_by_day_month_{}_{}.csv'.format(bbc_id, year))

In [40]:
res2 = res.groupby(['month']).agg('mean').drop(['day'], axis=1)
res2

,"Agriculture, animals, food and rural affairs","Asylum, immigration and nationality","Business, industry and consumers",Communities and families,"Crime, civil law, justice and rights","Culture, media and sport",Defence,Economy and finance,Education,Employment and training,Energy and environment,European Union,Health services and medicine,Housing and planning,International affairs,"Parliament, government and politics",Science and technology,Social security and pensions,Social services,Transport,Others
month,,,,,,,,,,,,,,,,,,,,,
1,41.641435,NaN,50.642097,25.875,26.302610,38.103328,33.375000,48.818116,40.875000,13.00,46.554135,44.363492,38.641901,NaN,43.550979,36.687504,NaN,NaN,NaN,36.738326,NaN
2,59.133333,NaN,58.637648,4.500,37.875588,46.374537,49.416667,64.460248,67.976190,NaN,53.238862,50.450289,40.601032,NaN,56.939197,46.692603,NaN,NaN,NaN,48.213770,NaN
3,67.677652,NaN,70.584944,58.000,37.595483,48.103507,74.174359,58.231037,59.906250,10.00,81.086275,49.118116,52.972161,NaN,59.480880,49.229228,NaN,NaN,NaN,44.326831,NaN
4,46.658730,NaN,45.061183,NaN,24.212295,33.420603,55.908333,77.154167,52.125000,NaN,37.053333,36.839286,40.589762,NaN,43.556229,32.808896,NaN,NaN,NaN,40.180500,NaN
5,63.322917,NaN,75.300536,2.500,39.678600,40.739858,31.302083,58.545679,44.951389,86.00,59.000000,32.351768,42.228241,NaN,47.125748,39.404774,NaN,NaN,NaN,42.080103,NaN
6,40.408333,NaN,37.449552,NaN,30.835825,30.214774,40.836667,46.278274,28.000000,NaN,60.416667,25.560642,39.039916,NaN,34.499998,28.760865,NaN,NaN,NaN,40.261189,NaN
7,83.626263,NaN,55.246289,NaN,37.005960,38.336024,40.203125,42.833333,52.777778,NaN,43.340909,34.727431,47.780392,NaN,38.341509,37.736216,NaN,NaN,NaN,35.565367,NaN
8,44.301587,NaN,51.408895,NaN,37.603158,41.840251,46.236111,57.933690,47.535714,NaN,53.437037,66.850741,42.302652,NaN,44.796060,41.588704,NaN,NaN,NaN,40.008164,NaN
9,62.301205,NaN,71.539323,NaN,55.948094,62.913203,54.902047,72.707283,62.535204,5.00,53.152652,71.466667,50.769566,NaN,68.185141,63.304750,NaN,NaN,NaN,68.753040,NaN


In [41]:
res2.to_csv('partition_topic_ranks_by_month_{}_{}.csv'.format(bbc_id, year))

# Method2: Word ranks weighing

In [378]:
topics = list(topics_name_to_index_map.keys())
header = ['day', 'month']
header += topics
rows = []
lrows = []

In [379]:
def score(start_pos, end_pos):
#     print(start_pos, end_pos)
    x = np.sum(range(start_pos, end_pos))/(end_pos - start_pos + 1)
    return x

In [380]:
for month in range(1, 13):
    for day in range(1, calendar.monthrange(year, month)[1]): # calendar.monthrange(year, month)[1]
        date_string = '{}-{}-{}'.format(day, calendar.month_abbr[month].lower(), year)
        print(date_string)
        df_day = df.loc[df.date == date_string].reset_index(drop=True)
        partition_ids = df_day.partition_id.unique()
        topic_ranks_for_day = {topic:[] for topic in topics}
        lengths_by_topic = {topic: [] for topic in topics}
        
        for id_ in partition_ids:
            df_partition = df_day.loc[df_day.partition_id == id_].reset_index(drop=True)
#             display(df_partition)
            length = df_partition['length'].sum()
            last_topic = None
            start_pos = 1
            end_pos = 1
            total_length_now = 0
            
            for index, row in df_partition.iterrows():
                
                if total_length_now >= 500:
                    break
                    
                if last_topic is None:
                    start_pos = 1
                    end_pos = start_pos + row['length'] - 1
                    last_topic = row['topic']
                elif last_topic == row['topic']:
                    end_pos = end_pos + row['length']
                    if index == len(df_partition) - 1:
                        topic_ranks_for_day[last_topic].append(score(start_pos, end_pos)) 
                        lengths_by_topic[last_topic].append(end_pos - start_pos + 1)
                else:
                    topic_ranks_for_day[last_topic].append(score(start_pos, end_pos))
                    lengths_by_topic[last_topic].append(end_pos - start_pos + 1)
                    start_pos = end_pos + 1
                    end_pos = start_pos + row['length'] - 1
                    last_topic = row['topic']
                total_length_now += row['length']
#                 print('start: ', start_pos)
#                 print('end: ', end_pos)
#         for topic in topics:
#             print(topic, topic_ranks_for_day[topic])
        row = [day, month]
        lrow = [day, month]
        lrow += [np.mean(lengths_by_topic[topic]) for topic in topics]
        row += [np.mean(topic_ranks_for_day[topic]) for topic in topics]
        rows.append(row)
        lrows.append(lrow)

1-jan-2016
2-jan-2016
3-jan-2016
4-jan-2016
5-jan-2016
6-jan-2016
7-jan-2016
8-jan-2016
9-jan-2016
10-jan-2016
11-jan-2016


/home/ubuntu/anaconda3/envs/econ/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/ubuntu/anaconda3/envs/econ/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


12-jan-2016
13-jan-2016
14-jan-2016
15-jan-2016
16-jan-2016
17-jan-2016
18-jan-2016
19-jan-2016
20-jan-2016
21-jan-2016
22-jan-2016
23-jan-2016
24-jan-2016
25-jan-2016
26-jan-2016
27-jan-2016
28-jan-2016
29-jan-2016
30-jan-2016
1-feb-2016
2-feb-2016
3-feb-2016
4-feb-2016
5-feb-2016
6-feb-2016
7-feb-2016
8-feb-2016
9-feb-2016
10-feb-2016
11-feb-2016
12-feb-2016
13-feb-2016
14-feb-2016
15-feb-2016
16-feb-2016
17-feb-2016
18-feb-2016
19-feb-2016
20-feb-2016
21-feb-2016
22-feb-2016
23-feb-2016
24-feb-2016
25-feb-2016
26-feb-2016
27-feb-2016
28-feb-2016
1-mar-2016
2-mar-2016
3-mar-2016
4-mar-2016
5-mar-2016
6-mar-2016
7-mar-2016
8-mar-2016
9-mar-2016
10-mar-2016
11-mar-2016
12-mar-2016
13-mar-2016
14-mar-2016
15-mar-2016
16-mar-2016
17-mar-2016
18-mar-2016
19-mar-2016
20-mar-2016
21-mar-2016
22-mar-2016
23-mar-2016
24-mar-2016
25-mar-2016
26-mar-2016
27-mar-2016
28-mar-2016
29-mar-2016
30-mar-2016
1-apr-2016
2-apr-2016
3-apr-2016
4-apr-2016
5-apr-2016
6-apr-2016
7-apr-2016
8-apr-2016
9-apr-

In [381]:
res = pd.DataFrame(rows, columns=header)
res = res.fillna(0)
res.to_csv('partition_topic_ranks_by_day_month_words_weight_{}_{}.csv'.format(bbc_id, year))
res

,day,month,"Agriculture, animals, food and rural affairs","Asylum, immigration and nationality","Business, industry and consumers",Communities and families,"Crime, civil law, justice and rights","Culture, media and sport",Defence,Economy and finance,Education,Employment and training,Energy and environment,European Union,Health services and medicine,Housing and planning,International affairs,"Parliament, government and politics",Science and technology,Social security and pensions,Social services,Transport,Others
0,1,1,0.0,0.0,0.0,0.0,0.000000,147.938974,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,139.741489,324.611383,0.0,0.0,0.0,141.000000,0.0
1,2,1,0.0,0.0,0.0,0.0,0.000000,84.057797,0.0,322.459119,0.0,0.0,0.0,0.0,0.000000,0.0,160.808824,109.417415,0.0,0.0,0.0,0.000000,0.0
2,3,1,0.0,0.0,0.0,0.0,0.000000,125.325271,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,35.307692,82.282828,0.0,0.0,0.0,0.000000,0.0
3,4,1,0.0,0.0,0.0,0.0,0.000000,157.062500,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,201.428571,71.478873,0.0,0.0,0.0,0.000000,0.0
4,5,1,0.0,0.0,0.0,0.0,198.245474,208.880642,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,204.989489,166.333278,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349,26,12,0.0,0.0,0.0,0.0,0.000000,133.333333,0.0,0.000000,0.0,0.0,0.0,0.0,184.682819,0.0,49.325581,352.333333,0.0,0.0,0.0,0.000000,0.0
350,27,12,0.0,0.0,0.0,0.0,0.000000,106.477011,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,80.729637,0.0,0.0,0.0,334.697674,0.0
351,28,12,0.0,0.0,0.0,0.0,0.000000,83.214216,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,55.462963,74.629630,0.0,0.0,0.0,79.071429,0.0
352,29,12,0.0,0.0,0.0,0.0,0.000000,119.515503,0.0,0.000000,0.0,0.0,0.0,0.0,239.481481,0.0,36.000000,149.867221,0.0,0.0,0.0,0.000000,0.0


In [382]:
for topic in topics:
    res[topic] = 500 - res[topic]

In [383]:
res2 = res.groupby(['month']).agg('mean').drop(['day'], axis=1)
res2

,"Agriculture, animals, food and rural affairs","Asylum, immigration and nationality","Business, industry and consumers",Communities and families,"Crime, civil law, justice and rights","Culture, media and sport",Defence,Economy and finance,Education,Employment and training,Energy and environment,European Union,Health services and medicine,Housing and planning,International affairs,"Parliament, government and politics",Science and technology,Social security and pensions,Social services,Transport,Others
month,,,,,,,,,,,,,,,,,,,,,
1,486.543561,500.0,482.399102,500.0,466.139098,375.142546,498.733333,489.251363,500.0,500.0,500.0,500.000000,495.539200,500.0,417.519570,392.870824,500.0,500.0,500.0,461.226809,500.0
2,500.000000,500.0,476.426871,500.0,486.328761,417.559227,486.428571,495.402695,500.0,500.0,500.0,479.591986,474.789248,500.0,442.100923,416.611567,500.0,500.0,500.0,471.472074,500.0
3,500.000000,500.0,500.000000,500.0,491.122558,451.743067,500.000000,493.608527,500.0,500.0,500.0,500.000000,500.000000,500.0,479.370591,461.618364,500.0,500.0,500.0,496.900000,500.0
4,500.000000,500.0,500.000000,500.0,500.000000,500.000000,500.000000,500.000000,500.0,500.0,500.0,500.000000,500.000000,500.0,500.000000,500.000000,500.0,500.0,500.0,500.000000,500.0
5,500.000000,500.0,476.508825,500.0,483.414656,403.720525,484.796636,474.257331,500.0,500.0,500.0,479.306260,481.654459,500.0,436.686465,425.145837,500.0,500.0,500.0,491.555971,500.0
6,500.000000,500.0,475.394614,500.0,465.074600,365.650275,500.000000,488.674465,500.0,500.0,500.0,454.411253,500.000000,500.0,463.652015,371.505228,500.0,500.0,500.0,481.654330,500.0
7,500.000000,500.0,494.356974,500.0,479.546765,428.328517,492.928092,500.000000,500.0,500.0,500.0,494.200000,500.000000,500.0,464.138868,426.250050,500.0,500.0,500.0,481.346977,500.0
8,500.000000,500.0,483.140876,500.0,477.556171,403.795689,498.800000,500.000000,500.0,500.0,500.0,492.117647,500.000000,500.0,429.157931,395.410840,500.0,500.0,500.0,500.000000,500.0
9,491.650831,500.0,481.311399,500.0,482.947340,399.163238,500.000000,500.000000,500.0,500.0,500.0,500.000000,475.821134,500.0,436.295105,402.335411,500.0,500.0,500.0,477.028361,500.0


In [384]:
res2.to_csv('partition_topic_ranks_by_month_words_weight_{}_{}.csv'.format(bbc_id, year))

In [385]:
res = pd.DataFrame(lrows, columns=header)
res

,day,month,"Agriculture, animals, food and rural affairs","Asylum, immigration and nationality","Business, industry and consumers",Communities and families,"Crime, civil law, justice and rights","Culture, media and sport",Defence,Economy and finance,Education,Employment and training,Energy and environment,European Union,Health services and medicine,Housing and planning,International affairs,"Parliament, government and politics",Science and technology,Social security and pensions,Social services,Transport,Others
0,1,1,NaN,NaN,NaN,NaN,NaN,150.428571,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,159.50,70.000000,NaN,NaN,NaN,283.0,NaN
1,2,1,NaN,NaN,NaN,NaN,NaN,53.833333,NaN,159.0,NaN,NaN,NaN,NaN,NaN,NaN,136.00,61.250000,NaN,NaN,NaN,NaN,NaN
2,3,1,NaN,NaN,NaN,NaN,NaN,51.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52.00,42.500000,NaN,NaN,NaN,NaN,NaN
3,4,1,NaN,NaN,NaN,NaN,NaN,30.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,189.00,71.000000,NaN,NaN,NaN,NaN,NaN
4,5,1,NaN,NaN,NaN,NaN,120.666667,92.400000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,196.75,51.222222,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349,26,12,NaN,NaN,NaN,NaN,NaN,267.666667,NaN,NaN,NaN,NaN,NaN,NaN,227.0,NaN,43.00,15.000000,NaN,NaN,NaN,NaN,NaN
350,27,12,NaN,NaN,NaN,NaN,NaN,71.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65.333333,NaN,NaN,NaN,258.0,NaN
351,28,12,NaN,NaN,NaN,NaN,NaN,20.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66.00,27.000000,NaN,NaN,NaN,42.0,NaN
352,29,12,NaN,NaN,NaN,NaN,NaN,51.000000,NaN,NaN,NaN,NaN,NaN,NaN,54.0,NaN,73.00,54.800000,NaN,NaN,NaN,NaN,NaN


In [386]:
res2 = res.groupby(['month']).agg('mean').drop(['day'], axis=1)
res2

,"Agriculture, animals, food and rural affairs","Asylum, immigration and nationality","Business, industry and consumers",Communities and families,"Crime, civil law, justice and rights","Culture, media and sport",Defence,Economy and finance,Education,Employment and training,Energy and environment,European Union,Health services and medicine,Housing and planning,International affairs,"Parliament, government and politics",Science and technology,Social security and pensions,Social services,Transport,Others
month,,,,,,,,,,,,,,,,,,,,,
1,176.0,NaN,162.222222,NaN,166.666667,64.775183,77.0,159.000000,NaN,NaN,NaN,NaN,53.666667,NaN,143.289098,61.333791,NaN,NaN,NaN,193.80,NaN
2,NaN,NaN,127.000000,NaN,120.333333,53.201389,153.0,167.000000,NaN,NaN,NaN,274.50,237.000000,NaN,112.431373,59.947619,NaN,NaN,NaN,140.25,NaN
3,NaN,NaN,NaN,NaN,102.250000,54.242424,NaN,86.000000,NaN,NaN,NaN,NaN,NaN,NaN,86.666667,95.861111,NaN,NaN,NaN,187.00,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,225.500000,NaN,196.000000,67.166253,138.5,176.333333,NaN,NaN,NaN,258.50,82.000000,NaN,124.858333,69.500610,NaN,NaN,NaN,78.00,NaN
6,NaN,NaN,163.250000,NaN,135.200000,73.797016,NaN,311.000000,NaN,NaN,NaN,220.75,NaN,NaN,131.216667,114.318103,NaN,NaN,NaN,190.50,NaN
7,NaN,NaN,141.000000,NaN,162.166667,62.900000,159.0,NaN,NaN,NaN,NaN,349.00,NaN,NaN,98.222222,75.326087,NaN,NaN,NaN,144.50,NaN
8,NaN,NaN,208.500000,NaN,134.250000,64.595333,73.0,NaN,NaN,NaN,NaN,68.00,NaN,NaN,143.470588,57.919753,NaN,NaN,NaN,NaN,NaN
9,278.0,NaN,188.666667,NaN,247.666667,64.984127,NaN,NaN,NaN,NaN,NaN,NaN,222.333333,NaN,117.159091,70.892667,NaN,NaN,NaN,154.50,NaN


In [387]:
res.to_csv('average_partition_lengths_{}_{}.csv'.format(bbc_id, year))
res2.to_csv('average_partition_lengths_month_{}_{}.csv'.format(bbc_id, year))